In [1]:
%load_ext autoreload
%autoreload 2
from econml._ortho_learner import _crossfit

In [3]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
class Wrapper:
    def __init__(self, model):
        self._model = model
    def fit(self, X, y, W=None):
        self._model.fit(X, y)
        return self
    def predict(self, X, y, W=None):
        return self._model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(5000, 3))
y = X[:, 0] + np.random.normal(size=(5000,))
folds = list(KFold(2).split(X, y))
model = Lasso(alpha=0.01)
nuisance, model_list, fitted_inds = _crossfit(Wrapper(model),
                                 folds,
                                 X, y, W=y, Z=None)
print(nuisance)
print(model_list)
fitted_inds

(array([-1.1057289 , -1.53756637, -2.4518278 , ...,  1.10628792,
       -1.82966233, -1.78227335]),)
[<__main__.Wrapper object at 0x12fd9d438>, <__main__.Wrapper object at 0x118e252b0>]


array([   0,    1,    2, ..., 4997, 4998, 4999])

In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._ortho_learner import _OrthoLearner
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y
    def fit(self, Y, T, W=None):
        self._model_t.fit(W, T)
        self._model_y.fit(W, Y)
        return self
    def predict(self, Y, T, W=None):
        return Y - self._model_y.predict(W), T - self._model_t.predict(W)
class ModelFinal:
    def __init__(self):
        return
    def fit(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        self.model = LinearRegression().fit(T_res.reshape(-1, 1), Y_res)
        return self
    def predict(self, X=None):
        return self.model.coef_[0]
    def score(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        return np.mean(Y_res - self.model.coef_[0]*T_res)**2
np.random.seed(123)
X = np.random.normal(size=(100, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(size=(100,))
est = _OrthoLearner(ModelNuisance(LinearRegression(), LinearRegression()), ModelFinal(),
                    n_splits=2, discrete_treatment=False, random_state=None)
est.fit(y, X[:, 0], W=X[:, 1:])
est.effect()

array([1.23440172])

In [14]:
est.score(y, X[:, 0], W=X[:, 1:])

0.0003880489502537651

In [15]:
from econml.dml import LinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
import numpy as np
X = np.random.normal(size=(100000, 3))
y = X[:, 0] + np.random.normal(size=(100000,))
est = LinearDMLCateEstimator(model_y=LinearRegression(), model_t=LinearRegression())
est.fit(y, X[:, 0], X[:, [1]], X[:, 2:], inference='statsmodels')
print(X[:1])
print(est.effect(X[:1, [1]]))
print(est.effect_interval(X[:1, [1]]))
print(est.coef_)
print(est.coef__interval())
print(est.const_marginal_effect(X[:1, [1]]))

[[ 1.0613513   0.0760928  -0.53482069]]
[0.99780349]
(array([0.99255731]), array([1.00304966]))
[ 0.99804439 -0.00316585]
(array([ 0.99281255, -0.00838755]), array([1.00327622, 0.00205585]))
[0.99780349]


In [16]:
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, MultiTaskLassoCV
import numpy as np
X = np.random.normal(size=(5000, 10))
y = X[:, 0] + np.random.normal(size=(5000,))
est = SparseLinearDMLCateEstimator(model_y=MultiTaskLassoCV(cv=3), model_t=LassoCV(cv=3),
                                   model_final=MultiTaskLassoCV(cv=3, fit_intercept=False))
dx = 8
est.fit(np.hstack([y.reshape(-1,1), y.reshape(-1,1)]), X[:, 0], X[:, 1:dx], X[:, dx:])
print(X[:1])
print(est.effect(X[:1, 1:dx]))
print(est.model_final.coef_)
print(est.const_marginal_effect(X[:1, 1:dx]))

[[-0.88666276  0.14006212  0.72897065  1.41017893  0.31083606  0.32659312
   0.53522834  1.21111218 -0.74004013 -0.6584672 ]]
[[0.99937549 0.99937549]]
[[9.99291159e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.71306507e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.99291159e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.71306507e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[0.99937549 0.99937549]]
